In [1]:
import torch
import torch.nn.utils.prune as prune 
import numpy as np
import plotly.express as px
import plotly.graph_objects as go 
import pandas as pd 
import torchvision
from torch.autograd import Variable
import os
from datetime import datetime, timedelta
import yfinance as yf


In [2]:
check_gpu = torch.cuda.is_available()
check_gpu

True

In [3]:
list_devices = torch.cuda.device_count()
list_devices

1

In [4]:
#os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
print(torch.__version__)

1.13.0.dev20220805


In [5]:
check_devices = torch.cuda.get_device_name()
check_devices

/home/austin/anaconda3/envs/work_x/lib/python3.8/site-packages/torch/cuda/__init__.py:150: UserWarning: 
NVIDIA GeForce RTX 3070 Ti Laptop GPU with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the NVIDIA GeForce RTX 3070 Ti Laptop GPU GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


'NVIDIA GeForce RTX 3070 Ti Laptop GPU'

In [6]:
crypto_name = 'apeusd'
#example_ohlc_df = pd.DataFrame({"Time": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Open": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"High": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Low": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],"Close": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})

ohlc_df = pd.read_csv('historical_price/' + crypto_name + '.csv')
ohlc_df.index = pd.to_datetime(ohlc_df['time'], unit = 'ms')

#ohlc_df.index = ohlc_df.index.tz_localize('UTC').tz_convert('US/Eastern')

In [7]:
print(len(ohlc_df))
history_used = int(len(ohlc_df)*0.8)

137126


In [8]:
price_dataset = ohlc_df.close[-history_used:]
time_index = ohlc_df.index[-history_used:]
price_dataset.tail()


time
2022-07-19 10:16:00    5.7765
2022-07-19 10:18:00    5.7755
2022-07-19 10:19:00    5.7596
2022-07-19 10:20:00    5.7585
2022-07-19 10:22:00    5.7676
Name: close, dtype: float64

In [9]:
x_data_len = int(history_used*.6)
train_cutoff = int(history_used*.8)
test_cutoff = int(history_used*.9)
x_data_len, train_cutoff, test_cutoff

(65820, 87760, 98730)

In [10]:
price_dataset_x = price_dataset[x_data_len:]
time_x = time_index[x_data_len:]
price_dataset_y = price_dataset[x_data_len:train_cutoff]
time_y = time_index[x_data_len:train_cutoff]
print(len(price_dataset_y))

21940


In [11]:
price_test_x = price_dataset[train_cutoff:test_cutoff]
time_test_x = time_index[train_cutoff:test_cutoff]
price_test_y = price_dataset[test_cutoff:]
time_test_y = time_index[test_cutoff:]
#price_test_val = price_dataset[900:1000]
#time_test_val = time_index[900:1000]

In [12]:
seq_len = 35
batch_size = 55

In [13]:
class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, seq_len=5):
        self.X = torch.tensor(X, dtype = torch.float32)
        self.y = torch.tensor(y, dtype = torch.float32)
        self.seq_len = seq_len
    def __len__(self):
        return self.X.__len__() - (self.seq_len - 1)

    def __getitem__(self, index):
        #print(len(self.y))
        X = self.X[index:index+self.seq_len]
        try:
            y = self.y[index+self.seq_len]
        except:
            y = self.y[self.seq_len]
        return X, y

In [14]:
train_dataset = TimeSeriesDataset(price_dataset_x, price_dataset_y, seq_len=seq_len)
test_dataset = TimeSeriesDataset(price_test_x, price_test_y, seq_len=seq_len)
#val_dataset = TimeSeriesDataset(price_test_val, price_test_y, seq_len=seq_len)

In [15]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    drop_last = True, 
    shuffle=False)

test_loader = torch.utils.data.DataLoader(
    test_dataset, 
    batch_size=batch_size, 
    drop_last=True,
    shuffle=False)

In [16]:
class LSTM_Model(torch.nn.Module):
    def __init__(self, input_dim , hidden_size , num_layers, batch_size):
        super(LSTM_Model, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_dim
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.lstm = torch.nn.LSTM(input_size=input_dim , hidden_size = hidden_size , num_layers= num_layers )
        self.fc = torch.nn.Linear(hidden_size,1)

    def forward(self,x,hn,cn):
        out , (hn,cn) = self.lstm(x , (hn,cn))
        final_out = self.fc(out[-1])
        return final_out,hn,cn

    def predict(self,x):
        hn,cn  = self.init()
        final_out = self.fc(hn[-1])
        return final_out

    def init(self):
        h0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        c0 =  torch.zeros(self.num_layers , self.batch_size , self.hidden_size)
        return h0 , c0


#device = torch.device('cpu')
#print(device)

input_dim = 1
hidden_size = seq_len
num_layers = 5
crypto_price_model = LSTM_Model(input_dim, hidden_size, num_layers, batch_size)


crypto_price_model#.to(device)

loss_function = torch.nn.MSELoss() # Criterion, I believe CrossEntropyLoss has SoftMax built-in so I used it here, this is why my last layer doesn't have a softmax
optimizer  = torch.optim.Adam(crypto_price_model.parameters(), lr=0.017) # used adaptive moment estimation to optimize the model

In [17]:
#training loop
def train(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.train()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(seq_len,batch_size,-1),hn,cn)
        out = out.view(-1)
        # print(out.shape)
        # print(y.shape)
        loss = loss_function(out.reshape(batch_size) , y)
        hn = hn.detach()
        cn = cn.detach()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f'Training Loss: {loss.item()}')
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"Train loss: {loss:>7f} ")
        loss_list.append(loss)
        predictions.append(out.detach().numpy())
        return predictions, loss_list
        

In [18]:
def test(dataloader, model):
    predictions = []
    loss_list = []
    hn , cn = model.init()
    model.eval()
    for batch , item in enumerate(dataloader):
        x , y = item
        y = y.type(torch.FloatTensor)
        #x = x.to(device)
        #y = y.to(device)
        out , hn , cn = model(x.reshape(seq_len,batch_size,1),hn,cn)
        loss = loss_function(out.reshape(batch_size) , y)
        print(f"test loss: {loss.item():>7f} ")
        if batch == len(dataloader)-1:
            loss = loss.item()
            print(f"Test loss: {loss:>7f} ")
        predictions.append(out.detach().numpy())
        loss_list.append(loss)
        return predictions, loss_list

In [19]:
epochs = 100
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}: ")
    train(train_loader, crypto_price_model)
test_predictions = test(test_loader, crypto_price_model)
#print(test_predictions[:][:][:][0])

Epoch 1: 
Training Loss: 37.44157409667969
Epoch 2: 
Training Loss: 33.38417434692383
Epoch 3: 
Training Loss: 21.858642578125
Epoch 4: 
Training Loss: 14.783547401428223
Epoch 5: 
Training Loss: 9.338367462158203
Epoch 6: 
Training Loss: 5.160794258117676
Epoch 7: 
Training Loss: 2.384399652481079
Epoch 8: 
Training Loss: 0.7801345586776733
Epoch 9: 
Training Loss: 0.0775514617562294
Epoch 10: 
Training Loss: 0.06413363665342331
Epoch 11: 
Training Loss: 0.46153008937835693
Epoch 12: 
Training Loss: 0.9648578763008118
Epoch 13: 
Training Loss: 1.3656305074691772
Epoch 14: 
Training Loss: 1.5699354410171509
Epoch 15: 
Training Loss: 1.5617610216140747
Epoch 16: 
Training Loss: 1.3791753053665161
Epoch 17: 
Training Loss: 1.0860875844955444
Epoch 18: 
Training Loss: 0.7508354783058167
Epoch 19: 
Training Loss: 0.4351493716239929
Epoch 20: 
Training Loss: 0.1872035413980484
Epoch 21: 
Training Loss: 0.03867312893271446
Epoch 22: 
Training Loss: 0.0010269524063915014
Epoch 23: 
Training L

In [20]:
forecast_price = np.array([num for num in test_predictions[0][0]]).flatten()
print(forecast_price)

[6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133
 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133
 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133
 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133
 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133
 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133
 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133 6.114133]


In [21]:
print(len(test_predictions[0]))
print()
def ave_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
#print(ave_percentage_error(price_test_y, forecast_price))

fig = go.Figure(layout_yaxis_range=[0,max(forecast_price+1) ])
pred_line = go.Scatter(x=time_test_y[:len(forecast_price)], y=forecast_price, mode = 'lines', name = 'Predicted Price')
fig.add_trace(pred_line)

fig.add_trace(go.Scatter(x=time_test_y[:len(forecast_price)], y=price_test_x[:len(forecast_price)], mode = 'lines', name = 'Actual Price'))

fig.update_layout(xaxis_title="Time", yaxis_title="Actual Price ($)", title=f"Predicted Price vs Actual Price for '{crypto_name}'")

#fig.write_json('LSTM_figures/'+f"{fig_name}_{crypto_name}_predicted_price_test.json")
fig.show()



1



In [22]:

def forecast_prices(most_recent_time, most_recent_data):
    fig = go.Figure(layout_yaxis_range=[0,max(forecast_price+1) ])
    forecast = crypto_price_model.predict(most_recent_data).detach().numpy()
    forecast = np.array([num for num in forecast]).flatten()
    td = most_recent_time + timedelta(minutes=len(forecast))
    #print(td)
    forecast_time = [td + timedelta(minutes=i) for i in range(len(forecast))]
    
    pred_line = go.Scatter(x=forecast_time, y=forecast, mode = 'lines', name = 'Predicted Price')

    fig.add_trace(pred_line)
    fig.update_layout(xaxis_title="Time", yaxis_title="Price ($)", title=f"Forecasted Price for '{crypto_name}'")
    #fig.show()
    return fig


In [23]:
def save_figure_as_json(fig, fig_name):
    if not os.path.exists('LSTM_figures'):
        os.mkdir('LSTM_figures')
    fig.write_json('LSTM_figures/'+f'{fig_name}_{crypto_name}.json')
save_figure_as_json(fig, 'test_predictions')

In [24]:
now = datetime.now().replace(second = 0, microsecond=0)
print(now)
ape = yf.download('APE3-USD', period = '2h', interval = '1m')


2022-08-05 10:06:00
[*********************100%***********************]  1 of 1 completed


In [25]:
#print(ape.columns)
#print(ape.index)
ape.index = ape.index.tz_convert('UTC').tz_convert('US/Central')
ape_price = ape['Close']
last_time = ape.index[-1]
print(last_time)
figure = forecast_prices(last_time,ape_price)
save_figure_as_json(figure, 'Forecast')
figure

2022-08-05 10:03:00-05:00


In [26]:
if not os.path.exists('LSTM_parameters'):
    os.mkdir('LSTM_parameters')
#fig.write_html('LSTM_parameters/'+f'{crypto_name}_predicted_price.html')
torch.save(crypto_price_model.state_dict(), f'LSTM_parameters/{crypto_name}_model_weights.pth')

In [27]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
crypto_price_model.load_state_dict(torch.load(f'LSTM_parameters/{crypto_name}_model_weights.pth',))

cuda


<All keys matched successfully>

In [28]:
avax_ticker = yf.Ticker('AVAX-USD')



In [29]:
avax_ticker.info

{'companyOfficers': [],
 'twitter': '"https://twitter.com/AvalancheAVAX"',
 'name': 'Avalanche',
 'startDate': 1594598400,
 'description': 'Avalanche (AVAX) is a cryptocurrency launched in 2020. Avalanche has a current supply of 404,229,626.49901325 with 284,428,387.85652924 in circulation. The last known price of Avalanche is 23.47793828 USD and is down -1.59 over the last 24 hours. It is currently trading on 291 active market(s) with $434,711,879.66 traded over the last 24 hours. More information can be found at https://avax.network/.',
 'maxAge': 1,
 'exchange': 'CCC',
 'shortName': 'Avalanche USD',
 'exchangeTimezoneName': 'UTC',
 'exchangeTimezoneShortName': 'UTC',
 'isEsgPopulated': False,
 'gmtOffSetMilliseconds': '0',
 'quoteType': 'CRYPTOCURRENCY',
 'symbol': 'AVAX-USD',
 'messageBoardId': 'finmb_AVAX_CCC',
 'market': 'ccc_market',
 'previousClose': 23.348064,
 'regularMarketOpen': 23.348064,
 'twoHundredDayAverage': 53.532074,
 'trailingAnnualDividendYield': None,
 'payoutRat